In [2]:
import os
import requests
import unicodedata
import re
import time

In [3]:
cwd = os.getcwd()
# path = os.path.dirname(os.path.dirname(cwd))
path = '/home/nacho/Documents/stjjalisco_sentencias_webscraping'
os.chdir(path)

In [4]:
os.getcwd()

'/home/nacho/Documents/stjjalisco_sentencias_webscraping'

In [5]:
#carpeta de descarga del dataset
folder_location = 'dataset_stjjalisco_sentencias'
if not os.path.exists(folder_location):os.mkdir(folder_location)
os.chdir(folder_location)
os.getcwd()

'/home/nacho/Documents/stjjalisco_sentencias_webscraping/dataset_stjjalisco_sentencias'

In [6]:
def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

In [7]:
page_number = 0
url = "https://publica-sentencias-backend.stjjalisco.gob.mx/tocas?page=" + str(page_number) + "&sala_id=&numero=&periodo=&derivado=&delito_id=&materia_id=&nombre_delito=&accion_especifica_id=&accion_simultanea_id="
page = requests.get(url)

In [8]:
page.json()['data']['tocas'].keys()

dict_keys(['current_page', 'data', 'first_page_url', 'from', 'last_page', 'last_page_url', 'next_page_url', 'path', 'per_page', 'prev_page_url', 'to', 'total'])

In [9]:
for toca in page.json()['data']['tocas']['data']:
    print(toca['id'], toca['sala_id'], toca['magistrado_id'], toca["materia_data"]["nombre"], toca["delito_data"]["nombre"])

41279 5 13 Civil ACCIÓN DE NULIDAD
41280 5 13 Civil ACCIÓN DE RESPONSABILIDAD CIVIL OBJETIVA
41281 5 13 Civil ACCIÓN DE USUCAPIÓN
41282 5 13 Mercantil ACCIÓN DE NULIDAD
41283 5 13 Civil ACCIÓN RESCISIÓN DE CONTRATO DE COMPRAVENTA
41284 5 13 Civil ACCIÓN DE TERMINACIÓN DE CONTRATO DE ARRENDAMIENTO
41285 5 13 Civil ACCIÓN PROFORMA
41286 5 13 Civil OTRAS
41287 5 13 Mercantil ACCIÓN CAMBIARIA DIRECTA
41288 5 13 Familiar DIVORCIO
41289 5 13 Mercantil ACCIÓN CAMBIARIA DIRECTA
41290 5 13 Familiar DIVORCIO
41291 5 13 Mercantil ACCIÓN CAMBIARIA DIRECTA
41292 5 13 Civil TERCERIA EXCLUYENTE DE DOMINIO
41293 5 13 Familiar DIVORCIO


In [10]:
toca = page.json()['data']['tocas']
print(toca['data'][0])

{'id': 41279, 'sala_id': 5, 'magistrado_id': 13, 'user_id': 15, 'tipo_juicio_id': 1, 'sentido_id': 2, 'tipo_sentencia': 1, 'resolucion': None, 'materia_id': 1, 'delito_id': 929, 'accion_especifica_id': None, 'accion_simultanea_id': None, 'numero': 291, 'periodo': 2022, 'derivado': None, 'file': 'sala_5/41279.pdf', 'fecha_publicacion': '21-06-2022', 'perspectiva_genero': 2, 'principio_ninez': 2, 'menores': None, 'created_at': '2022-06-21 09:15:21', 'updated_at': '2022-06-21 09:15:21', 'deleted_at': None, 'salas_data': {'id': 5, 'nombre': 'QUINTA SALA ESPECIALIZADA EN MATERIA CIVIL', 'created_at': '2021-01-16 09:35:53', 'updated_at': '2021-01-16 09:35:53', 'deleted_at': None}, 'materia_data': {'id': 1, 'nombre': 'Civil', 'created_at': '2021-05-13 16:16:27', 'updated_at': '2021-05-13 16:16:27', 'deleted_at': None}, 'tipo_juicio_data': {'id': 1, 'materia_id': 1, 'nombre': 'CIVIL ORDINARIA', 'created_at': '2021-05-13 16:16:27', 'updated_at': '2021-05-13 16:16:27', 'deleted_at': None, 'mater

In [11]:
page_number = 0
url = "https://publica-sentencias-backend.stjjalisco.gob.mx/tocas?page=" + str(page_number) + "&sala_id=&numero=&periodo=&derivado=&delito_id=&materia_id=&nombre_delito=&accion_especifica_id=&accion_simultanea_id="
page = requests.get(url)

In [ ]:
last_page = page.json()['data']['tocas']['last_page']
for page_number in range(0, last_page + 1):
    try:
        page = requests.get("https://publica-sentencias-backend.stjjalisco.gob.mx/tocas?page=" + str(page_number))
        if page.status_code == 429:
            time.sleep(int(page.headers['retry-after']))
        else:
            for toca in page.json()['data']['tocas']['data']:
                try:
                    print(str(toca['id']) + str("_") + str(toca['sala_id']) + str("_") + str(toca['magistrado_id']) + str("_") + str(toca["fecha_publicacion"]) + str("_") + str(toca["materia_data"]["nombre"]) + str("_") + str(toca["delito_data"]["nombre"]))
                    if toca["delito_data"]["nombre"] == 'ACCIÓN DE NULIDAD':
                    # if toca["delito_data"]["nombre"] == 'ACCIÓN DE NULIDAD' and toca["materia_data"]["nombre"] == 'Civil':
                        toca_file_response = requests.get("https://publica-sentencias-backend.stjjalisco.gob.mx/toca/" + str(toca["id"]) + "/file")
                        if toca["accion_especifica_id"] is None and toca["accion_simultanea_id"] is None:
                            toca_file_filename = 'id-' + str(toca['id']) + str('_') + 'salaid-'+ str(toca['sala_id']) + str('_') + 'magistradoid-' + str(toca['magistrado_id']) + str('_') + str(toca["fecha_publicacion"]) + str("_") + str(toca["materia_data"]["nombre"]) + str("_") + str(toca["delito_data"]["nombre"])
                        elif toca["accion_especifica_id"] is not None and toca["accion_simultanea_id"] is not None:
                            toca_file_filename = 'id-' + str(toca['id']) + str('_') + 'salaid-' + str(toca['sala_id']) + str('_') + 'magistradoid-' + str(toca['magistrado_id']) + str('_') + str(toca["fecha_publicacion"]) + str("_") + str(toca["materia_data"]["nombre"]) + str("_") + str(toca["delito_data"]["nombre"]) + str("_") + str(toca["accion_especifica_id"]['nombre']) + str("_") + str(toca["accion_simultanea_id"]['nombre'])
                        elif toca["accion_especifica_id"] is not None and toca["accion_simultanea_id"] is None:
                            toca_file_filename = 'id-' + str(toca['id']) + str('_') + 'salaid-'+ str(toca['sala_id']) + str('_') + 'magistradoid-' + str(toca['magistrado_id']) + str('_') + str(toca["fecha_publicacion"]) + str("_") + str(toca["materia_data"]["nombre"]) + str("_") + str(toca["delito_data"]["nombre"]) + str("_") + str(toca["accion_especifica_id"]['nombre'])
                        elif toca["accion_especifica_id"] is None and toca["accion_simultanea_id"] is not None:
                            toca_file_filename = 'id-' + str(toca['id']) + str('_') + 'salaid-'+ str(toca['sala_id']) + str('_') + 'magistradoid-' + str(toca['magistrado_id']) + str('_') + str(toca["fecha_publicacion"]) + str("_") + str(toca["materia_data"]["nombre"]) + str("_") + str(toca["delito_data"]["nombre"]) + str("_") + str(toca["accion_simultanea_id"]['nombre'])
                except TypeError:
                    pass
                finally:
                    with open(slugify(toca_file_filename) + '.pdf', 'wb') as f:
                        f.write(toca_file_response.content)   

    except:
        print("Connection refused by the server..")
        time.sleep(5)
        print("continue...")
        continue